In [3]:
import pandas as pd
import numpy as np

In [4]:
sf_data = pd.read_csv("/scratch/public/kushk/sf_with_homeloc.csv")


In [5]:
print("Original size: ", len(sf_data))

Original size:  47723945


In [6]:
#homedropped = len(sf_data[sf_data["home_tract"].isna()])
#print("Number of tracts without home_tract: " ,  homedropped)

NameError: name 'homedropped' is not defined

In [ ]:
#tractdropped = (sf_data[sf_data["tract"].isna()]) 
#print("Number of tracts without tract: " , tractdropped )

In [7]:
sf_data = sf_data[~sf_data["home_tract"].isna()]
sf_data = sf_data[~sf_data["tract"].isna()]
sf_data = sf_data[["date", "tract" , "home_tract"]]

In [8]:
print("Final size after dropping nulls: ", len(sf_data))

Final size after dropping nulls:  47653523


In [9]:
nn_data = pd.read_csv("/scratch/public/kushk/sfbay_13county_nearest_neigbbor_new.csv")

In [10]:
nn_data.head()

,SRC_GEOID,NBR_GEOID,LENGTH,NODE_COUNT
0,6001400100,6001404300,11.573519,0
1,6001400100,6001404400,4424.828741,0
2,6001400100,6001421600,477.810654,0
3,6001400100,6001422600,1282.939488,0
4,6001400100,6001422700,553.272946,0


In [11]:
sf_data.head()

,date,tract,home_tract
0,2012-07-26T00:49:46Z,6.081613e+09,6.085509e+09
1,2012-07-26T00:50:45Z,6.067008e+09,6.067008e+09
2,2012-07-26T00:50:50Z,6.095253e+09,6.095253e+09
3,2012-07-26T00:50:56Z,6.081601e+09,6.081603e+09
4,2012-07-26T00:50:58Z,6.085501e+09,6.085501e+09


In [12]:
neighbors = {}
for src,dest in zip(nn_data["SRC_GEOID"] , nn_data["NBR_GEOID"]):
    if src not in neighbors:
        neighbors[src] = set()
    neighbors[src].add(dest)

In [13]:
def processNeighbors(row):
    tract = int(row["tract"])
    home_tract = int(row["home_tract"])
    if(tract == home_tract):
        return "home"
    elif(home_tract in neighbors and tract in neighbors[home_tract]):
        return "neighbor"
    else:
        return "non-neighbor"

In [14]:
%%time
sf_data["locality"] = sf_data.apply(processNeighbors, axis=1)
sf_data["locality"].value_counts()

CPU times: user 40min, sys: 1min 36s, total: 41min 37s
Wall time: 41min 45s


In [15]:
sf_data = sf_data[sf_data["locality"] !=  "not in set?"]

In [16]:
sf_data["locality"].value_counts()

home            28990907
non-neighbor    14944888
neighbor         3717728
Name: locality, dtype: int64

In [17]:
sf_data.head()

,date,tract,home_tract,locality
0,2012-07-26T00:49:46Z,6.081613e+09,6.085509e+09,non-neighbor
1,2012-07-26T00:50:45Z,6.067008e+09,6.067008e+09,home
2,2012-07-26T00:50:50Z,6.095253e+09,6.095253e+09,home
3,2012-07-26T00:50:56Z,6.081601e+09,6.081603e+09,non-neighbor
4,2012-07-26T00:50:58Z,6.085501e+09,6.085501e+09,home


In [18]:
overall = sf_data.groupby(["tract" , "locality" ]).size().reset_index().rename(columns={0:'count'})

In [19]:
sf_data['hour'] = sf_data['date'].str[11:13]
dayvalues = ['04', '05', '06', '07', '08', '09', '10', '11', '12', '13','14', '15', '16', '17', '18']
sf_data['day'] = sf_data.hour.map(lambda x: 1 if x in dayvalues else 0)

In [20]:
dayNight = sf_data.groupby(["tract" , "locality", "day"]).size().reset_index().rename(columns={0:'count'})
dayNight.head()

,tract,locality,day,count
0,6.001400e+09,home,0,719
1,6.001400e+09,home,1,753
2,6.001400e+09,neighbor,0,288
3,6.001400e+09,neighbor,1,120
4,6.001400e+09,non-neighbor,0,2409


In [21]:
sf_data['date_helper'] = sf_data['date'].str[0:10]
sf_data['weekday'] = sf_data.date_helper.map(lambda x: 0 if pd.to_datetime(x).weekday() > 4 else 1)


In [22]:
weekDay = sf_data.groupby(["tract" , "locality", "weekday"]).size().reset_index().rename(columns={0:'count'})
weekDay.head()

,tract,locality,weekday,count
0,6.001400e+09,home,0,378
1,6.001400e+09,home,1,1094
2,6.001400e+09,neighbor,0,131
3,6.001400e+09,neighbor,1,277
4,6.001400e+09,non-neighbor,0,1733


In [23]:
tracts = list(set(sf_data["tract"]))

#sf_data./

In [24]:
%%time
counts = [[] , [] , []]
dayCounts = [[[], []] , [[],[]] , [[],[]]]
weekDayCounts = [[[], []] , [[],[]] , [[],[]]]
locs = ["home" , "neighbor" , "non-neighbor"]


for tract in tracts:
    one_tract = overall[overall["tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        if(len(one_loc) > 0):
            counts[i].append(one_loc["count"].iloc[0])
        else:
            counts[i].append(0)
        i+=1
        
for tract in tracts:
    one_tract = dayNight[dayNight["tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        for val in [0,1]:
            one_val = one_loc[one_loc["day"] == val]
            if(len(one_val) > 0):
                dayCounts[i][val].append(one_val["count"].iloc[0])
            else:
                dayCounts[i][val].append(0)
        i+=1
                         
for tract in tracts:
    one_tract = weekDay[weekDay["tract"] == tract]
    i=0
    for loc in locs:
        one_loc = one_tract[one_tract["locality"] == loc]
        for val in [0,1]:
            one_val = one_loc[one_loc["weekday"] == val]
            if(len(one_val) > 0):
                weekDayCounts[i][val].append(one_val["count"].iloc[0])
            else:
                weekDayCounts[i][val].append(0)
        i+=1

CPU times: user 56 s, sys: 420 ms, total: 56.4 s
Wall time: 56.5 s


In [25]:
sf_final = pd.DataFrame({"tract": pd.Series(tracts)})
i=0
for loc in locs:
    sf_final[loc] = pd.Series(counts[i])
    sf_final[loc + " day"] = pd.Series(dayCounts[i][1])
    sf_final[loc + " night"] = pd.Series(dayCounts[i][0])
    sf_final[loc + " weekday"] = pd.Series(weekDayCounts[i][1])
    sf_final[loc + " weekend"] = pd.Series(weekDayCounts[i][0])
    i+=1

In [26]:
sf_final

,tract,home,home day,home night,home weekday,home weekend,neighbor,neighbor day,neighbor night,neighbor weekday,neighbor weekend,non-neighbor,non-neighbor day,non-neighbor night,non-neighbor weekday,non-neighbor weekend
0,6.067003e+09,3931,2419,1512,3011,920,182,97,85,122,60,3031,1533,1498,1978,1053
1,6.013354e+09,1065,532,533,810,255,153,99,54,105,48,10105,4624,5481,7517,2588
2,6.013354e+09,5051,3015,2036,3649,1402,439,178,261,288,151,5281,2362,2919,3699,1582
3,6.095252e+09,7348,4390,2958,5353,1995,460,302,158,320,140,1965,1188,777,1197,768
4,6.075015e+09,3857,2140,1717,2611,1246,1135,484,651,680,455,9941,3750,6191,6118,3823
5,6.075023e+09,13025,8760,4265,8854,4171,539,306,233,384,155,9169,5285,3884,5581,3588
6,6.001410e+09,12986,7699,5287,9914,3072,793,442,351,518,275,3813,2027,1786,2501,1312
7,6.081602e+09,6977,3580,3397,5197,1780,2239,1121,1118,1575,664,11827,4030,7797,7510,4317
8,6.081602e+09,11347,6927,4420,8220,3127,1575,814,761,1044,531,6680,3388,3292,4436,2244
9,6.081602e+09,7751,4793,2958,5889,1862,3525,1189,2336,2479,1046,24598,7155,17443,15694,8904


In [27]:
sf_final.to_csv("Absolute_new_1024_13county_nodrop.csv")

In [ ]:
sum(sf_final["home"]) + sum(sf_final["non-neighbor"])